# Kubernetes Basics

<p align=center><a href=https://kubernetes.io/docs/concepts/overview/what-is-kubernetes/><img src=images/k8s.png width=500></a></p>


> *Kubernetes* is an open-source platform that focuses on declarative configuration and automation for managing containerized workloads.

In order to run multiple containers, a long `docker run` command or `docker compose` (particularly useful in this scenario) is required. However, when an application that needs to run tens or hundreds of containers is deployed, the health and working condition of every container must be ensured, which is a difficult task. This is where Kubernetes (`k8s`) thrives. 

Kubernetes orchestrates containers. It is capable of spinning up many containers in a particular configuration. Additionally, if a container goes down for any reason, Kubernetes can initiate a new one as a replacement. This is extremely important for critical applications.

## Kubernetes Benefits

Kubernetes has many benefits and features, such as:

- **Multiple-container Deployment**: Kubernetes enables the deployment of multiple containers across a cluster
    - The user chooses which and how many containers should be deployed
    - The user also specifies the number of container replicas for fault tolerance <br><br>

- **Self-healing**: If a container is down, i.e. not responding to health checks, Kubernetes will spin up a replacement

- **Autoscaling**: If the current containers are not enough to process all the requirements, Kubernetes can deploy more containers within the cluster. Conversely, if the resources are in excess, Kubernetes can destroy some to maintain the required balance.

- **Easily Discoverable**: Each container can be accessed by its name (or IP) and port

- **Load Balancing**: If a single node is overloaded by high traffic, Kubernetes distributes the load among several nodes to ensure high availability and efficient resource utilization

- **Storage Orchestration**: Kubernetes allows us to mount storage volumes, whether they are specific to a node, shared across multiple nodes, or of another type. This capability ensures that your application can save and retrieve data reliably.

## Kubernetes Components

Kubernetes encompasses many concepts, each essential for efficiently working with the platform. In this section we will provide an overview of the components that form the foundation of a Kubernetes cluster.

When Kubernetes is deployed, it creates a cluster with the components illustrated in the diagram below:

![](./images/components-of-kubernetes.svg)

### High-level components overview

- *Nodes*: 

  - *Nodes* are the **worker machines** in a Kubernetes cluster. These can be physical or virtual servers.
  - **Nodes** are responsible for running the containers that make up your applications. They provide the computing resources (CPU, memory, storage) necessary to execute these containers.
  - Each node runs a Kubernetes runtime, such as Docker to manage and run containers <br><br>

- *Pods*: 

  - **Pods** are the smallest deployable units in Kubernetes, and they are used to encapsulate one or more containers
  - **Pods** are scheduled to run on nodes by the Kubernetes *control plane*. When you create a pod, the Kubernetes scheduler decides which node in the cluster should run that pod based on various factors like resource availability, node health, and scheduling policies.
  - All containers within the same pod share the same network namespace, which means they can communicate with each other over `localhost` <br><br>

- **Control Plane**: The **Control Plane** is a set of components that collectively manage and control the state of the cluster. It's responsible for making global decisions about the cluster (e.g., which nodes should run which pods), detecting and responding to cluster events, and maintaining the desired state of all resources.

> For learning purposes, we will begin by working with a single Node on one physical machine. However, in production environments, the Control Plane spans multiple machines to increase fault tolerance, and can contain a large number Nodes, often numbering in the thousands, to meet the demands of complex applications.

### Control Plane Components

The **Control Plane** in Kubernetes serves as the brain of the cluster, responsible for managing and maintaining the desired state of the entire system. Think of it as the central control hub that governs the behavior and functionality of the cluster. Its primary tasks include:

- **Receiving Commands**: The Control Plane receives commands and requests from users, administrators, and applications. These commands are typically sent through the *`kube-apiserver`*, which acts as the entry point to the Kubernetes API.

- **Cluster State Management**: It continuously monitors and manages the cluster's desired state. This includes tracking the configuration of objects like Pods, *Services*, and *Deployments*, and ensuring that the actual state matches the desired state. We will learn in more detail about Services and Deployments later.

- **Scheduling Work**: The Control Plane is responsible for determining where and how containers should run within the cluster. It uses the *`kube-scheduler`* to make decisions about which nodes should host specific Pods.

- **Scaling and Healing**: If there are discrepancies between the desired and actual states, the Control Plane takes corrective actions. For instance, if a Pod fails, it initiates the creation of a replacement Pod to ensure that the desired state is maintained.

- **Coordination**: It coordinates various tasks and components within the cluster to ensure seamless communication and cooperation among Nodes, Pods, and other resources

#### 1. `kube-apiserver`

> The `kube-apiserver` serves as the gateway to your Kubernetes cluster, exposing the Kubernetes API. This API enables communication with and management of the cluster's resources.

Methods of interactions include:

- **HTTP Requests**: You can interact with the `kube-apiserver` using HTTP requests, as it provides a RESTful API. This method is extensively used for its flexibility and accessibility. 

- **Command Line Tools**: Kubernetes offers command-line tools like `kubectl` for interacting with an existing Kubernetes cluster, and `kubeadm` for *cluster bootstrap operations*. Cluster bootstrap operations refer to the process of creating a new Kubernetes cluster from scratch. This involves setting up the control plane components, initializing the cluster, and preparing the nodes that will participate in the cluster.  

> Using command-line tools is a preferred approach due to its readability, automation benefits, and compatibility with various Kubernetes API versions.

#### 2. `etcd`

> *`etcd`* serves as the reliable storage backend for Kubernetes, storing critical data that defines the entire cluster's configuration and state.

Key features include:

- **Local Data Storage**: `etcd` stores data locally on the control plane node(s), ensuring quick access and reliability

- **Cluster State Preservation**: It's important to note that if all control plane nodes fail, the cluster state is lost, necessitating the recreation of the entire infrastructure. Hence, regular backups of `etcd` data to an external storage device are recommended.

#### 3. `kube-scheduler`

> The *`kube-scheduler`* plays a crucial role in the cluster by handling the assignment of newly created Pods to specific Nodes.

The Node selection criteria include:

- **Resource Requirements**: The scheduler considers the resource requirements specified for each Pod

- **Policies and Constraints**: It applies policies and constraints to ensure optimal placement

- **Data Locality**: It optimizes data locality, efficiently scheduling Pods to reduce data transfer overhead

#### 4. `kube-controller-manager`

> The *`kube-controller-manager`* runs various controller processes, each responsible for maintaining specific aspects of the cluster's desired state.

These processes continually watch the cluster through the `kube-apiserver` and work to align the current state with the desired state. For instance, the node controller responds when a node goes down, ensuring the cluster remains in the desired configuration.

#### 5. `cloud-controller-manager`

Kubernetes can be deployed on various cloud providers, such as AWS, Azure, Google Cloud. These cloud environments offer a wide range of resources and services that Kubernetes can leverage to enhance the functionality and resilience of containerized applications. 

> The *`cloud-controller-manager`* is a specialized component within the Kubernetes Control Plane designed to interact with and manage these cloud-specific resources. Similar to the `kube-controller-manager`, the `cloud-controller-manager` is responsible for maintaining the desired state of these cloud resources.

For example, the `cloud-controller-manager` may include controllers like the *node controller*, which queries the cloud provider to assess the health and status of nodes in the cluster. It makes decisions based on cloud-specific information, ensuring that the cluster can effectively adapt to changing conditions, such as instances being terminated or network configurations being adjusted.

### Node Components

Each node in a Kubernetes cluster consists of different components that important crucial roles in managing containers and ensuring the cluster's functionality:

#### 1. `kubelet`

> The *`kubelet`* component is responsible for ensuring the proper execution of containers within Kubernetes pods. It achieves this by adhering to the *`PodSpec`* (Pod Specification), which defines how each pod should be configured and run.

The `PodSpec` includes key details like container definitions, resource requirements, volume mounts, environment variables, service accounts, labels, annotations, and security settings. We will learn how to configure `PodSpec` in a later lesson.

The `kubelet` reads the `PodSpec` for each pod from the API server and ensures that the containers operate in line with the specifications. If any discrepancies or issues arise, it takes corrective actions, such as restarting containers, to maintain the desired pod state.

#### 2. `kube-proxy`

> The *`kube-proxy`* manages network communication within a Kubernetes cluster. It configures network rules, ensuring bidirectional communication between Pods, load balancing for services, and enforcing network policies for security. This component plays an important role in maintaining network connectivity and reliability in the cluster.

#### 3. Container Runtime

> The *container runtime* is the software responsible for executing containers. Kubernetes is compatible with various container runtimes. Some common options include:

- **Docker**: Docker is a widely used container runtime known for its versatility and comprehensive features
- **containerd**: Containerd is a simpler container runtime, offering a lightweight alternative to Docker

### Interactions Between Control Plane and Node Components

In a Kubernetes cluster, control plane components and node components work together to maintain cluster stability, manage resources, and ensure seamless communication. Here's an overview of their interactions:

1. `kube-apiserver` Communication:

  - The `kube-apiserver` serves as the central API endpoint for control plane components
  - Node components, such as `kubelet` and `kube-proxy`, interact with it to access cluster information and synchronize the cluster state

2. `etcd` Data Store:

  - `etcd` stores the cluster's configuration and state
  - Both control plane components and node components read and update cluster information in `etcd` as needed

3. `kube-scheduler` Node Selection:

  - The `kube-scheduler` uses data from `kube-apiserver` to make informed decisions about where to place new Pods
  - It communicates its decisions to `kubelet` on the chosen Node, which then initiates the deployment of Pods, guided by the `PodSpec`

4. Controller Managers for State Maintenance:

  - Controller processes in `kube-controller-manager` continuously monitor the cluster state via `kube-apiserver`
  - They ensure that the cluster's current state aligns with the desired state, taking corrective actions when needed

5. Node Components and Pod Management:

  - `kubelet` on each Node ensures the proper execution and health of containers within Pods, adhering to the `PodSpec`
  - `kube-proxy` manages network rules on Nodes, enabling bidirectional network communication between Pods within the cluster and external entities

6. Container Runtimes for Container Execution:

  - Node components rely on the chosen container runtime (e.g., Docker, containerd) to execute containers within Pods

## Kubernetes Objects

> In Kubernetes, *objects* are the building blocks that define and maintain the desired state of the cluster. They serve as persistent entities representing what the cluster should look like.

### Creating Kubernetes Objects

Kubernetes offers two primary approaches to manage objects:

- **Declarative Commands**: Declarative commands specify the desired state of the cluster, describing what resources should exist and how they should behave. Kubernetes then takes care of ensuring the actual state matches this desired state. Declarative commands are favored for their predictability and consistency.

- **Imperative Commands**: In contrast, imperative commands directly instruct Kubernetes to perform specific actions, such as creating, deleting, or updating resources. While imperative commands offer immediate control, they may require more manual intervention and can lead to *configuration drift* over time.

Configuration drift refers to the unintended and gradual deviation between the desired configuration of a system and its actual state. This drift is more likely to occur with imperative commands because they can lead to untracked and immediate changes, making it challenging to maintain the intended configuration over time.

> In this section, we'll explore how to create these objects using declarative commands. 

Kubernetes objects allow us to specify:

- Which applications to deploy and where they should run within the cluster
- The desired behavior of applications, including restart policies, scaling, upgrades, and more
- Allocation of resources available to applications

To create object, a request must be made to the Kubernetes API via the `kubelet-apiserver`. While it's possible to send direct requests, we'll leverage `kubectl` for most operations. However, before we request the creation of resources, we need to define the object's specifications.

### Describing Kubernetes Objects

Kubernetes operates as a foundational platform upon which various projects and applications are built. Think of it as a "programming language for deployment." In Kubernetes, we embrace *declarative programming*, where we specify the desired outcome rather than detailing the individual steps required to achieve it.

> Kubernetes uses declarative programming to describe the state of the entire system.

<br>

<details>
    <summary> <font size=+1>Click here to see the difference between imperative and declarative programming.</font> </summary>

<br>

> Using imperative programming (or configuration), we define each necessary step to achieve the desired outcome. Examples include:

- Create variable `x`, and assign it the value of `1`
- Scrape data from a website, and retrieve an appropriate `<image>` tag
- Pass the image through a function to calculate the probability that it contains a logo

> On the other hand, using declarative programming (or configuration), we describe the desired state of a system without specifying the required steps. Examples include:

- I want variable `x` with the value of `1` assigned to it
- I want an `<image>` tag taken out of this website
- I want the probability that the image contains a logo

</details>

Since Kubernetes exposes a RESTful API, we can represent objects using `JSON` descriptions. However, this approach, although supported, can be challenging to maintain.

> An alternative, more human-readable approach is to use `YAML` to define object specifications and allow `kubectl` to seamlessly convert these representations into `JSON` format.

Below, we provide an example of an object configuration along with a description of its fields:

``` yaml
# configuration file stored in deployment_example.yaml

apiVersion: apps/v1
kind: Deployment
metadata:
  # Unique key of the Deployment instance
  name: deployment-example
spec:
  # 3 Pods should exist at all times.
  replicas: 3
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        # Apply this label to pods and default
        # the Deployment label selector to this value
        app: nginx
    spec:
      containers:
      - name: nginx
        # Run this image
        image: nginx:1.14
```

Let's break down the components of this `YAML` file:

- `apiVersion`: Specifies the API version for the Kubernetes resource being defined. In this case, `apps/v1`, which is the API version for Deployments, a type of Kubernetes object. Don't worry about not knowing the different types of Kubernetes objects and their differences yet, as we will cover this extensively in the next lesson.
- `kind`: Specifies the type of object, which is a Deployment in this case
- `metadata`: Provides metadata for the object, including the name, which is set to `deployment-example`

> To determine the appropriate API version based on the resources to be used, check out Kubernetes' documentation [here](https://kubernetes.io/docs/reference/kubernetes-api/). 

For example, in the declarative code above, a Deployment resource can be observed. In the documentation, you can check the Deployment API and determine that you would need the `apps/v1` API version.<br><br>

<p align=center><img src=images/Deployment.png height =450 width=850></p>

### `spec`

> In Kubernetes, the *`spec`* section serves as a critical component of object definitions. It outlines the desired characteristics and configurations of the resource. This information is provided by the user and informs Kubernetes how to manage the resource.

For instance, in the case of a Deployment object example above:

- The `spec `specifies that every Pod with the label `app: nginx` (with the name `nginx`), and classified as a `Deployment`, should be created and maintained

- It communicates that three *replica* Pods should run within the cluster to ensure redundancy and availability. When you specify `replicas`, you're essentially telling Kubernetes that you want multiple identical copies (replicas) of the same Pod to be running concurrently in the cluster. Running multiple replica Pods serves several purposes, one of which is redundancy and availability. If one of the Pods were to fail for any reason, the other replicas can continue to serve the application's workload. 

- It defines the blueprint for each Pod through a template:

  - The template assigns the label `app: nginx` to these Pods for easy identification
  - Within the template, it specifies that each Pod is created from a single container named `nginx`, enhancing discoverability
  - The image attribute within the template dictates the specific Docker image `nginx:1.14` to be used for each container

In the [API documentation](https://kubernetes.io/docs/reference/kubernetes-api/), you can view the available `spec`s for resources. For example, for the Deployment `spec` have a look [here](https://kubernetes.io/docs/reference/kubernetes-api/workload-resources/deployment-v1/).

<p align=center><img src=images/Deployment_Spec.png width=800 height=500></p>

In addition to `spec`, Kubernetes objects also include a `status` section to provide insights into the current state of the resource.

### `status`: Describing Current State

> The `status` section describes the current state of the resource. While the user defines the `spec`, Kubernetes internally updates the `status`. This distinction is vital as it allows users to query the real-time status of the deployment.

This allows controllers, which are responsible for maintaining the resource, to make informed decisions based on the status, therefor enabling them to steer objects closer to their desired state, ensuring reliability and performance.

### Desired Vs. Actual State

It's important to understand that clusters may not always precisely match the desired actual state. Kubernetes operates as a continuously running system, and while it tries to align with the desired state, certain factors can introduce disparities:

- Frequent updates to configuration files may introduce delays in the implementation of changes as Kubernetes processes these updates
- The propagation of changes may be delayed, especially in clusters with a high number of nodes, as Kubernetes coordinates these changes across the entire cluster
- Random node failures, though Kubernetes will automatically reinstantiate necessary Pods, may temporarily affect the actual state

### Creating Kubernetes Objects (Continuation)

Having described a Kubernetes object, the next step is to instruct the `kube-apiserver` to create it. This involves conveying your intentions for the object to Kubernetes. We will use the declarative approach for this.

> In the declarative approach, your focus is on specifying the configuration files, not the precise operation. Instead, you entrust the Kubernetes engine to automatically determine and execute the appropriate `create`, `update`, or `delete` operations based on the defined configurations.

Advantages of declarative configuration include:

- **Efficient Directory Management**: It's effortless to work with directories containing numerous configuration files. Changes applied to these files automatically affect the cluster.

- **Scalability**: Handling multiple configurations allows you to manage a broader range of objects within your cluster

- **Simplified Commands**: Only two commands are generally needed: `kubectl diff` to verify operations and `kubectl apply` to enact changes. We will look into detail at different `kubectl` commands and how to use them in the next section.

- **Selective Updates**: Instead of replacing the entire configuration, you can patch specific parts, reducing complexity and streamlining maintenance

Let's first install Kubernetes locally, and then we will proceed with the different commands you can use to interact with Kubernetes objects.

## Installing Kubernetes

To work with Kubernetes, you'll need to install two essential tools: `kubectl` and `minikube`. These tools enable you to manage and deploy Kubernetes resources on your local machine without the need for cloud resources. In this section, we'll guide you through the installation process for both tools.

### Installing `kubectl`

`kubectl` is the command-line tool used to interact with Kubernetes clusters and deploy applications. The installation steps vary depending on your operating system.

<details>
    <summary> For Linux Users </summary>

You can install `kubectl` on Linux using the following commands:

```
curl -LO "https://dl.k8s.io/release/$(curl -L -s https://dl.k8s.io/release/stable.txt)/bin/linux/amd64/kubectl"
curl -LO "https://dl.k8s.io/$(curl -L -s https://dl.k8s.io/release/stable.txt)/bin/linux/amd64/kubectl.sha256"
echo "$(<kubectl.sha256) kubectl" | sha256sum --check
sudo install -o root -g root -m 0755 kubectl /usr/local/bin/kubectl
```
If you encounter any error during the installation, refer to the [Kubernetes website](https://kubernetes.io/docs/tasks/tools/install-kubectl-linux/). 
</details>

<details>
    <summary> For Mac Users </summary>

On macOS, you can use `homebrew` for a straightforward installation:

```
brew install kubectl 
```
If you encounter any error during the installation, refer to the [Kubernetes website](https://kubernetes.io/docs/tasks/tools/install-kubectl-linux/).
</details>

<details>
    <summary> For Windows Users </summary>

To install `kubectl` on Windows, follow these steps:

1. Download the latest release with the following command:

```
curl.exe -LO "https://dl.k8s.io/release/v1.28.1/bin/windows/amd64/kubectl.exe"
```

Make sure to check for the latest release command [here](https://kubernetes.io/docs/tasks/tools/install-kubectl-windows/) as new versions come up quite often.

2. Move the directory containing `kubectl.exe` to a location in your `PATH`. To modify the `PATH` variable in Windows:

    - In the Windows Start menu, search for **Edit the system environment variables** and select it

    - In the **System Properties** window, click the **Environment Variables** button

    - Under the **System variables** section, scroll down and find the **Path** variable. Select it and click the **Edit** button.

    - In the **Edit Environment Variable** window, click **New** and then add the directory where `kubectl.exe` file has been downloaded (e.g., `C:\Program Files` if the file is located in `C:\Program Files`). Click **OK** to save your changes

    - Close all the open windows by clicking **OK** or **Apply** to save the changes


Once you've installed `kubectl`, regardless of your operating system, you can verify the installation by running the following command:

`kubectl version --client`

### Installing `minikube`

[`minikube`](https://minikube.sigs.k8s.io/docs/start/) is a powerful tool that simplifies the process of running Kubernetes from your local machine. It allows you to run a single-node Kubernetes cluster locally, providing an ideal environment for experimentation and learning. If you already have Docker installed on your local computer, you should be able to install Minikube with ease.

<details>
    <summary> For Linux Users </summary>

To install Minikube on Linux, open your terminal and run the following commands:

```
curl -LO https://storage.googleapis.com/minikube/releases/latest/minikube-linux-amd64
sudo install minikube-linux-amd64 /usr/local/bin/minikube
```

To start Minikube you will need to run `minikube start`. If you encounter any problems, you might need to run the following command: `sudo usermod -aG docker $USER && newgrp docker`.

If you are still unable to start Minikube, follow these commands:

```bash
minikube config set driver docker
minikube delete
minikube start --driver=docker
```

</details>

 <details>
    <summary> For Mac Users </summary>

Simply run the following commands in your terminal:
```
brew install minikube
```

</details>
<details>
    <summary> For Windows Users </summary>

For Windows users, follow these steps:

1. **Download the Minikube Installer**: Download the `.exe` file here: https://storage.googleapis.com/minikube/releases/latest/minikube-installer.exe

2. **Run PowerShell as an Administrator**: After installing it, open PowerShell as an Administrator to add the Minikube binary to your `PATH`. Run the following commands:

```
$oldPath = [Environment]::GetEnvironmentVariable('Path', [EnvironmentVariableTarget]::Machine)
if ($oldPath.Split(';') -inotcontains 'C:\minikube'){ `
  [Environment]::SetEnvironmentVariable('Path', $('{0};C:\minikube' -f $oldPath), [EnvironmentVariableTarget]::Machine) `
}
```
</details>

To confirm that Minikube was correctly installed, run `minikube --help`. You may need to close and open the terminal before running any command. 

### Configuring `minikube`

Regardless of your operating system, once Minikube is installed, you can configure it by running the following command: `minikube start`.

> This command initiates the creation of a Kubernetes configuration file, which informs Kubernetes (or `kubectl`) where to start deploying Kubernetes resources. If you intend to deploy resources to a cloud infrastructure, you can later modify this configuration file to suit your needs.

A *context* refers to a set of access parameters for a specific Kubernetes cluster. These parameters include the cluster's server location, authentication details, and the default *namespace* to use when interacting with the cluster. A **namespace** is a way to create isolated and logically separate clusters within the same physical Kubernetes cluster. The default namespace is the namespace where resources are created if you don't specify a namespace when creating a resource.

When you create a Kubernetes configuration file it can contain multiple contexts, each representing a different Kubernetes cluster. However, with Minikube the configuration file should only contain one context. You can switch between these contexts to specify which cluster you want to work with at any given time. This is particularly useful when you are managing multiple Kubernetes clusters.

To inspect the configuration context and see which context is currently active, you can use the following command:

In [2]:
!kubectl config get-contexts

CURRENT   NAME       CLUSTER    AUTHINFO   NAMESPACE
*         minikube   minikube   minikube   default


You should observe that we are indeed using the cluster provided by Minikube. To find this information in the configuration file (usually located in` ~/.kube`), you can confirm that it aligns with the details obtained:

<p align=center><img src=images/config_minikube.png width=500></p>

In the provided example, the namespace is set to `default`. By default, when you start Minikube or change the `kubeconfig` file, Kubernetes creates four namespaces. To view them, you can run the following command:

In [3]:
!kubectl get namespace

NAME              STATUS   AGE
default           Active   3m5s
kube-node-lease   Active   3m5s
kube-public       Active   3m6s
kube-system       Active   3m7s


To learn more about namespaces, you can visit the [Kubernetes documentation](https://kubernetes.io/docs/concepts/overview/working-with-objects/namespaces/). For now, understand that each namespace contains different resources. You can view, for example, the pods inside the `kube-system` namespace with the following command:

In [4]:
!kubectl -n kube-system get pods -o wide

NAME                               READY   STATUS    RESTARTS        AGE     IP             NODE       NOMINATED NODE   READINESS GATES
coredns-5d78c9869d-4nqdp           1/1     Running   0               3m45s   10.244.0.2     minikube   <none>           <none>
etcd-minikube                      1/1     Running   0               3m57s   192.168.49.2   minikube   <none>           <none>
kube-apiserver-minikube            1/1     Running   0               3m57s   192.168.49.2   minikube   <none>           <none>
kube-controller-manager-minikube   1/1     Running   1 (4m11s ago)   3m57s   192.168.49.2   minikube   <none>           <none>
kube-proxy-49fzl                   1/1     Running   0               3m46s   192.168.49.2   minikube   <none>           <none>
kube-scheduler-minikube            1/1     Running   0               3m57s   192.168.49.2   minikube   <none>           <none>
storage-provisioner                1/1     Running   1 (3m9s ago)    3m49s   192.168.49.2   minikube  

As mentioned earlier, Kubernetes automatically restarts resources that have gone down. To demonstrate this, you can delete a pod and observe its recovery:

In [5]:
!kubectl -n kube-system delete pod kube-proxy-49fzl 

pod "kube-proxy-49fzl" deleted


Afterward, when you check the pods again:

In [7]:
!kubectl -n kube-system get pods -o wide

NAME                               READY   STATUS    RESTARTS        AGE     IP             NODE       NOMINATED NODE   READINESS GATES
coredns-5d78c9869d-4nqdp           1/1     Running   0               4m56s   10.244.0.2     minikube   <none>           <none>
etcd-minikube                      1/1     Running   0               5m8s    192.168.49.2   minikube   <none>           <none>
kube-apiserver-minikube            1/1     Running   0               5m8s    192.168.49.2   minikube   <none>           <none>
kube-controller-manager-minikube   1/1     Running   1 (5m22s ago)   5m8s    192.168.49.2   minikube   <none>           <none>
kube-proxy-6jdt4                   1/1     Running   0               39s     192.168.49.2   minikube   <none>           <none>
kube-scheduler-minikube            1/1     Running   0               5m8s    192.168.49.2   minikube   <none>           <none>
storage-provisioner                1/1     Running   1 (4m20s ago)   5m      192.168.49.2   minikube  

You will notice that the resource is up and running once more.

Finally, Minikube provides a convenient dashboard for visualizing resources. You can access it by running the following command:

In [ ]:
minikube dashboard

The figure below displays the output after selecting `all namespaces` in the dropdown menu:

<p><img src=images/minikube_dashboard.png></p>

In subsequent lessons, we will explore how to create and manage these resources.

> Don't forget to stop Minikube once you are done using the following command: `minikube stop`.

## `kubectl` Commands

To work effectively with Kubernetes, it's essential to understand the syntax and usage of `kubectl` commands:

```
kubectl [command] [TYPE] [NAME] [flags]
```

Here's a breakdown of the components:

- `command`: Specifies the operation to be performed (e.g., `get`, `describe`)

- `TYPE`: Specifies the resource type. It is case-insensitive and supports abbreviations and plurals. For example, `kubectl get namespace` is equivalent to `kubectl get ns`, and `kubectl get pods` is equivalent to `kubectl get po`.

- `NAME`: Specifies the name of the resource. If omitted, it will return everything. For instance, `kubectl get pods` lists all pods in the cluster.

The most useful ones are provided below. For the full list, check [here](https://kubernetes.io/docs/reference/kubectl/overview/#in-cluster-authentication-and-namespace-overrides).

### `apply`

The `apply` command is used for configuring the cluster to your desired state by applying configuration changes defined in `YAML` or `JSON` files. It allows you to create, update, or delete resources based on the contents of these configurations files. Whether you're managing a local development setup with tools like Minikube or deploying to a production environment, `kubectl apply` helps you manage your Kubernetes resources effectively.

```bash
kubectl apply -f FILENAME [flags]
```
- `-f`: Specifies the filename or directory containing the configuration files that define the desired state of the cluster
- `[flags]`: Additional command flags and options that can be used to customize the behavior of the command

To apply the configuration changes defined in a file named `my-app.yaml` to your Kubernetes cluster, you can use the following command:

```bash
kubectl apply -f my-app.yaml
```
This command reads the contents of `my-app.yaml` and applies the specified resource definitions to the cluster, creating or updating resources as necessary to match the desired state.

Important flags include:

- `-R`: Recursively reads configuration files within a directory
- `-f`: Specifies the filename or directory to read configuration from

### `diff`

The `diff` command checks the changes that will be applied to the desired state of the cluster before actually applying those changes. It's a recommended practice to run this command as part of your workflow to ensure that you understand the impact of your configuration changes.

```
kubectl diff -f -R FILENAME [flags]
```
- `-f`: Specifies that you are providing a file (usually a `YAML` or `JSON` file) that contains the desired configuration changes
- `-R`: Recursively checks changes for resources defined within the provided file and its dependencies
- `FILENAME`: The path to the file containing the desired configuration changes
- `[flags]`: Additional command flags and options that can be used to customize the behavior of the command

To check and preview the changes that would be applied when you apply a configuration file named `my-app.yaml`, you can use the following command:

```bash
kubectl diff -f -R my-app.yaml
```
This command will compare the desired configuration in `my-app.yaml` with the current state of the cluster and provide you with a summary of the changes that would be made if you applied the configuration.

### `get`

The `get` command is used for listing one or more resources and providing basic information about Kubernetes objects. It offers a quick overview of the current state of resources, helping you manage and monitor your Kubernetes cluster effectively.

```bash
kubectl get (-f FILENAME | TYPE [NAME | /NAME | -l label]) [--watch] [--sort-by=FIELD] [[-o | --output]=OUTPUT_FORMAT] [flags]
```
- `(-f FILENAME | TYPE [NAME | /NAME | -l label])`: Specifies the resource you want to list. You can use a filename to list resources defined in a `YAML` file, or you can specify the resource type (e.g., pods, services) along with an optional name, name prefix, or label selector to target specific resources.
- `[--watch]`: Enables real-time monitoring of resource changes. The command will continuously display updates as resources change.
- `[--sort-by=FIELD]`: Sorts the resource list by the specified field
- `[[-o | --output]=OUTPUT_FORMAT]`: Specifies the desired output format for the resource list. You can choose from various output formats such as `JSON`, `YAML`, wide, or custom-columns.
- `[flags]`: Additional command flags and options that can be used to customize the behavior of the command

For example, to list all pods in the current namespace, you can use the following command:

```bash
kubectl get pods
```
Or to list services in a specific namespace, specify the namespace using the `-n` flag:

```bash
kubectl get services -n my-namespace
```

### `describe`

The `describe` command is used to inspect the state and details of one or more resources within your cluster. It provides an overview of essential information about resources, helping you understand their current configurations and conditions.

```bash
kubectl describe (-f FILENAME | TYPE [NAME_PREFIX | /NAME | -l label]) [flags]
```

- `(-f FILENAME | TYPE [NAME_PREFIX | /NAME | -l label])`: Specifies the resource you want to describe. You can use a filename to describe resources defined in a `YAML` file, or you can specify the resource type (e.g., pods, services) along with an optional name prefix, specific name, or label selector to target specific resources.
- `[flags]`: Additional command flags and options that can be used to customize the behavior of the command

For example, to display detailed information about a specific pod, use the following command:

```bash
kubectl describe pod POD_NAME
```
Replace `POD_NAME` with the name of the pod you want to describe.

Or to inspect the configuration and conditions of a service, you can use:

```bash
kubectl describe service SERVICE_NAME
```

Replace `SERVICE_NAME` with the name of the service you want to describe.

### `attach`

The `attach` command is a valuable tool for interacting with the standard input (stdin) and the standard output (stdout) of a container running within a pod. It is particularly useful for debugging purposes, allowing you to attach to a running container and interact with it.

```bash
kubectl attach POD -c CONTAINER [-i] [-t] [flags]
```

- `POD`: Specifies the name of the pod containing the container you want to attach to
- `-c`,` --container`: Specifies the name of the container within the pod to attach to. This is required, especially when the pod contains multiple containers.
- `-i`: Enables interactive mode, which allows you to interact with the container's stdin
- `-t`: Allocates a terminal, making it possible to have an interactive shell session within the container
- `[flags]`: Additional command flags and options that can be used to customize the behavior of the `kubectl attach` command

To attach a running container within a pod and open an interactive terminal session, use the following command:

```bash
kubectl attach POD -c CONTAINER -i -t
```

Replace `POD` with the name of the pod, and `CONTAINER` with the name of the container.

### `cp`

The `cp` command is a versatile utility that facilitates the copying of files to and from containers. It is particularly valuable for debugging and inspecting container disk outputs.

```bash
kubectl cp <file-spec-src> <file-spec-dest> [options]
```

- `<file-spec-src>`: Specifies the source file or directory within a container or a pod
- `<file-spec-dest>`: Specifies the destination file or directory on the local file system or within a container or a pod
- `[options]`: Additional command options that can be used to customize the behavior of the command

For example, if we wanted to copy a file named `example.txt` from a container to our local file system, use the following command:

```bash
kubectl cp <pod-name>:<path-to-file> ./example.txt
```

Replace `<pod-name>` with the name of the pod, and `<path-to-file>` with the path to the file within the container.

### `exec`

The `exec` command allows you to execute commands within a container. It is primarily used for debugging and troubleshooting purposes, providing insights into a container's current state and allowing you to interact with it directly.

```bash
kubectl exec POD [-c CONTAINER] [-i] [-t] [flags] [-- COMMAND [args...]]
```

- `POD`: Specifies the name of the pod in which you want to execute a command
- `-c`, `--container`: Optionally, you can specify the name of the container within the pod where you want to run the command. If not specified, it defaults to the first container in the pod.
- `-i`, `-t`: These flags indicate that you want to interact with the command, providing an interactive terminal session. `-i` enables standard input, and `-t` allocates a terminal.
- `COMMAND [args...]`: This is the actual command you want to run within the specified container

Let's look at an example where you start an interactive shell session in a pod. You can use the following command:

```bash
kubectl exec -it POD -- /bin/sh
```

Here you would have to replace `POD` with the name of your pod.

### `delete`

The `delete` command is used to delete resources. It provides a convenient way to clean up resources that are no longer needed or have become obsolete.

```bash
kubectl delete RESOURCE NAME [flags]
```

- `RESOURCE`: Specifies the type of resource you want to delete. This can be one of many Kubernetes resource types, such as pods, services, deployments, configmaps, and more.
- `NAME`: Specifies the name of the resource you want to delete

For example to delete a pod, you can use the following command:

```bash
kubectl delete pod my-pod
```

Common flags include:

- `-n`,` --namespace`: Specifies the namespace in which the resource is located. If not specified, the current context's namespace is used.
- `--grace-period`: Allows you to specify a grace period (in seconds) before forcefully deleting the resource
- `--force`: Forces deletion of the resource, even if it's not recommended (e.g., pods that are part of a controlled deployment)
- `--wait`: Waits for resources to be fully deleted before returning
- `--all`: Deletes all resources of the specified type in the current namespace

## Key Takeaways

- Kubernetes is an open-source platform for managing containerized workloads with a focus on declarative configuration and automation
- Kubernetes provides multiple-container deployment, self-healing, auto-scaling, discoverability, load balancing, and storage orchestration
- Kubernetes consists of nodes, pods, and a control plane
- Nodes are worker machines hosting containerized applications
- Pods are the smallest deployable units and consist of one or more containers
- The control plane manages nodes and pods, ensuring their desired state
- Kubernetes objects represent the desired state of the cluster
- `kubectl` is used to interact with Kubernetes objects
- `kubectl` commands have the format: `kubectl [command] [TYPE] [NAME] [flags]`. Common commands include `apply`, `diff`, `get`, `describe`, `attach`, `cp`, and `exec`.
- Namespaces provide a scope for resource names, ensuring uniqueness within a namespace. Kubernetes uses namespaces for organizing resources.

## Further Reading

- Check out [Kubernetes API Conventions](https://github.com/kubernetes/community/blob/master/contributors/devel/sig-architecture/api-conventions.md) for an in-depth understanding of Kubernetes Objects and the internal mechanisms of the project
- Explore the alternatives to `minikube`. Read up on `kind` and `k3s`, as well as their advantages and disadvantages
- Check out [Client Libraries](https://kubernetes.io/docs/reference/using-api/client-libraries/), which allow us to communicate with `kubelet-apiserver` via certain programming languages.
- Check out the [Python Client Library](https://github.com/kubernetes-client/python/), and explore the tasks that can be automated using it